In [1]:
import pandas as pd

https://www.gov.br/inep/pt-br/areas-de-atuacao/pesquisas-estatisticas-e-indicadores/censo-da-educacao-superior/resultados

### Colunas

- NU_ANO_CENSO: Ano do Censo

- NO_REGIAO: Nome da região geográfica do local de oferta do curso

- SG_UF: Sigla da Unidade da Federação do local de oferta do curso

- NO_MUNICIPIO: Nome do Município do local de oferta do curso

- CO_MUNICIPIO: Código do IBGE do municipio do local de oferta do curso

- IN_CAPITAL: Informa se o local de oferta do curso está localizado em capital de Estado
	- 0. Não
	- 1. Sim
	- (.) Não aplicável (Curso a distância)

- TP_REDE: Rede de Ensino
	- 1. Pública
	- 2. Privada

- NO_CINE_ROTULO: Nome do curso, conforme adaptação da Classificação Internacional Normalizada da Educação Cine/Unesco

- TP_GRAU_ACADEMICO: Tipo do grau acadêmico conferido ao ao aluno pela conclusão dos requisitos exigidos pelo curso
	- 1. Bacharelado
	- 2. Licenciatura
	- 3. Tecnológico
	- 4. Bacharelado e Licenciatura
	- (.) Não aplicável (cursos com nivel acadêmico igual a sequencial de formação específica ou  cursos de área básica de curso identificada pela variável TP_ATRIBUTO_INGRESSO)

- TP_MODALIDADE_ENSINO: Tipo da modalidade de ensino do curso
	- 1. Presencial
	- 2. Curso a distância

- TP_NIVEL_ACADEMICO: Tipo do nível acadêmico do curso
	- 1. Graduação
	- 2. Seqüencial de Formação Específica

- QT_ING: Quantidade de ingressante no curso

- QT_ING_FEM: Quantidade de ingressante do sexo feminino no curso

- QT_CONC: Quantidade de concluinte no curso

- QT_CONC_FEM: Quantidade de concluinte no curso do sexo feminino

In [2]:
path_cursos = 'data/cursos.csv.zip'
df_cursos = pd.read_csv(path_cursos)
df_cursos

,NU_ANO_CENSO,NO_REGIAO,SG_UF,NO_MUNICIPIO,CO_MUNICIPIO,IN_CAPITAL,TP_REDE,NO_CINE_ROTULO,TP_GRAU_ACADEMICO,TP_MODALIDADE_ENSINO,TP_NIVEL_ACADEMICO,QT_ING,QT_ING_FEM,QT_CONC,QT_CONC_FEM
0,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,1,Ciência da computação,1.0,1,1,90.0,14.0,42.0,4.0
1,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,3.0,1,1,56.0,4.0,0.0,0.0
2,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,3.0,1,1,95.0,25.0,1.0,1.0
3,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,1.0,1,1,37.0,12.0,11.0,2.0
4,2009,Centro-Oeste,DF,Brasília,5300108.0,1.0,2,Sistemas de informação,1.0,1,1,2.0,0.0,9.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92467,2020,NaN,NaN,NaN,NaN,NaN,2,Sistemas de informação,3.0,2,1,0.0,0.0,0.0,0.0
92468,2020,NaN,NaN,NaN,NaN,NaN,2,Jogos digitais,3.0,2,1,0.0,0.0,0.0,0.0
92469,2020,NaN,NaN,NaN,NaN,NaN,2,Sistemas de informação,3.0,2,1,0.0,0.0,0.0,0.0
92470,2020,NaN,NaN,NaN,NaN,NaN,1,Ciência de dados,1.0,2,1,0.0,0.0,0.0,0.0
